In [1]:
%pip install emoji

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import emoji 

In [3]:
emoji_dict = {  "0" : "\u2764\uFE0F" ,
                "1" : ":baseball:" ,
                "2" : ":grinning_face_with_big_eyes:" ,
                "3" : ":disappointed_face:" ,
                "4" : ":fork_and_knife:" 
             }

In [4]:
import pandas as pd 
import numpy as np 

In [5]:
train = pd.read_csv('train_emoji.csv' , header = None )

In [6]:
test = pd.read_csv('test_emoji.csv' , header = None ) 

In [7]:
data = train.values 
xtrain = train[0]
ytrain = train[1]
xtest = test[0]
ytest = test[1] 

In [8]:
for i in range ( 5 ):
    print( xtrain[i] , emoji.emojize( emoji_dict[str(ytrain[i])]) )

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


In [9]:
f = open('glove.6B.50d.txt' , encoding='utf-8') 

In [10]:
embed_ind = {}
for line in f :
    values = line.split()
    word = values[0]
    coefs = np.asarray( values[1:] , dtype = 'float' )
    embed_ind [word] = coefs 
f.close()

In [11]:
emb_dim = embed_ind["eat"].shape[0]

In [12]:
def embed_out( x ):
    maxlen = 10  
    out = np.zeros( (x.shape[0], maxlen , emb_dim ) )
    for ix in range ( x.shape[0] ):
        x[ix] = x[ix].split()   
        for ij in range ( len(x[ix]) ):
            try:
                out[ix][ij] = embed_ind[ x[ix][ij].lower() ]
            except:
                out[ix][ij] = np.zeros( (50 ,) )
                
    return out
embed_matrix_train = embed_out( xtrain )
embed_matrix_test = embed_out( xtest )

C:\Users\meena\AppData\Local\Temp\ipykernel_23852\2635502865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[ix] = x[ix].split()


In [13]:
print( xtrain[0] )
print( len(xtrain[0]) )

['never', 'talk', 'to', 'me', 'again']
5


In [14]:
%pip install keras.utils   
from keras.utils.np_utils import to_categorical  
ytrain = to_categorical ( ytrain , num_classes = 5 )
ytest = to_categorical( ytest , num_classes = 5 )

You should consider upgrading via the 'c:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
from keras.models import Sequential
from keras.layers import *

In [16]:
model = Sequential()
model.add( LSTM (64 , input_shape = (10,50),return_sequences = True ) )
model.add( Dropout(0.5) )
model.add( LSTM(64, return_sequences=False) )
model.add( Dropout(0.5) )
model.add( Dense(5) )
model.add( Activation ('softmax') )
model.compile( loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics= ['accuracy'] )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            29440     
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
 activation (Activation)     (None, 5)                 0         
                                                                 
Total params: 62,789
Trainable params: 62,789
Non-traina

In [17]:
hist = model.fit( embed_matrix_train , ytrain , epochs = 1000 , batch_size = 64 , shuffle = True , validation_split = 0.2  )

Epoch 1/1000
2/2 [==============================] - 9s 2s/step - loss: 1.5993 - accuracy: 0.2667 - val_loss: 1.6023 - val_accuracy: 0.2222
Epoch 2/1000
2/2 [==============================] - 0s 71ms/step - loss: 1.5523 - accuracy: 0.3238 - val_loss: 1.6214 - val_accuracy: 0.2593
Epoch 3/1000
2/2 [==============================] - 0s 77ms/step - loss: 1.5131 - accuracy: 0.3143 - val_loss: 1.6496 - val_accuracy: 0.3333
Epoch 4/1000
2/2 [==============================] - 0s 74ms/step - loss: 1.4822 - accuracy: 0.3429 - val_loss: 1.6877 - val_accuracy: 0.2593
Epoch 5/1000
2/2 [==============================] - 0s 73ms/step - loss: 1.4771 - accuracy: 0.3524 - val_loss: 1.7166 - val_accuracy: 0.2222
Epoch 6/1000
2/2 [==============================] - 0s 92ms/step - loss: 1.4480 - accuracy: 0.3810 - val_loss: 1.7228 - val_accuracy: 0.2222
Epoch 7/1000
2/2 [==============================] - 0s 81ms/step - loss: 1.4516 - accuracy: 0.3905 - val_loss: 1.7059 - val_accuracy: 0.2593
Epoch 8/1000
2/

In [18]:
model.evaluate( embed_matrix_test , ytest )

2/2 [==============================] - 0s 9ms/step - loss: 5.8260 - accuracy: 0.4464


[5.8260040283203125, 0.4464285671710968]

In [19]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model.h5" , monitor = 'accuracy' , verbose = True , save_best_only = True )
earlystop = EarlyStopping( monitor = 'val_acc' , patience = 10 )  
hist = model.fit( embed_matrix_train , ytrain , epochs = 1000 , batch_size = 64 , shuffle = True , validation_split = 0.2  )

Epoch 1/1000
2/2 [==============================] - 0s 74ms/step - loss: 2.1412e-04 - accuracy: 1.0000 - val_loss: 3.3813 - val_accuracy: 0.6667
Epoch 2/1000
2/2 [==============================] - 0s 59ms/step - loss: 3.2393e-04 - accuracy: 1.0000 - val_loss: 3.3809 - val_accuracy: 0.6667
Epoch 3/1000
2/2 [==============================] - 0s 44ms/step - loss: 4.7080e-04 - accuracy: 1.0000 - val_loss: 3.3807 - val_accuracy: 0.6667
Epoch 4/1000
2/2 [==============================] - 0s 44ms/step - loss: 2.7348e-04 - accuracy: 1.0000 - val_loss: 3.3823 - val_accuracy: 0.6667
Epoch 5/1000
2/2 [==============================] - 0s 43ms/step - loss: 2.3779e-04 - accuracy: 1.0000 - val_loss: 3.3848 - val_accuracy: 0.6667
Epoch 6/1000
2/2 [==============================] - 0s 42ms/step - loss: 1.6676e-04 - accuracy: 1.0000 - val_loss: 3.3876 - val_accuracy: 0.6667
Epoch 7/1000
2/2 [==============================] - 0s 41ms/step - loss: 1.6654e-04 - accuracy: 1.0000 - val_loss: 3.3898 - val_ac

In [20]:

model.evaluate( embed_matrix_test , ytest )

2/2 [==============================] - 0s 6ms/step - loss: 4.1134 - accuracy: 0.5000


[4.113440036773682, 0.5]